In [1]:
# Unified Router caller (same for all tests)
import requests, json

AGENT_URL = "http://127.0.0.1:8000/collect"  # ← change if different

def call_router(intent: str, args: dict | None = None):
    r = requests.post(AGENT_URL, json={"intent": intent, "args": args or {}}, timeout=60)
    r.raise_for_status()
    resp = r.json()
    print("OK:", resp.get("ok"), "| Intent:", resp.get("intent"))
    if not resp.get("ok"):
        print(json.dumps(resp, indent=2))
        raise SystemExit
    # Show which provider actually answered
    src = (resp.get("meta") or {}).get("source") or {}
    print("Provider(s):", src)
    # Return whole response so you can inspect raw as needed
    return resp

def show_first(obj, n=3):
    if isinstance(obj, list):
        for i, x in enumerate(obj[:n], 1):
            print(f"{i:>2}.", (x if isinstance(x, (str,int,float)) else {k:x.get(k) for k in list(x)[:8]}))
    elif isinstance(obj, dict):
        print({k: obj.get(k) for k in list(obj)[:12]})
    else:
        print(repr(obj))

In [2]:
# What it does: Lists soccer leagues; TSDB primary (falls back to AllSports if empty).
# Try editing:
#   name="English Premier League", "Spanish La Liga", "Italian Serie A"
#   country="England" (TSDB) or countryName="Italy" (AS resolver)

resp = call_router("leagues.list", {
    # "name": "English Premier League",
    # "country": "England",          # TSDB filter
    # "countryName": "Italy",        # AllSports resolver (to countryId)
})
data = resp["data"]
print("Keys:", list(data.keys()))
leagues = data.get("leagues") or data.get("result") or []
print("Total leagues:", len(leagues))
show_first(leagues, n=15)

OK: True | Intent: leagues.list
Provider(s): {'primary': 'allsports', 'fallback': None}
Keys: ['success', 'result']
Total leagues: 976
 1. {'league_key': 4, 'league_name': 'UEFA Europa League', 'country_key': 1, 'country_name': 'eurocups', 'league_logo': 'https://apiv2.allsportsapi.com/logo/logo_leagues/', 'country_logo': None}
 2. {'league_key': 1, 'league_name': 'UEFA European Championship', 'country_key': 1, 'country_name': 'eurocups', 'league_logo': None, 'country_logo': None}
 3. {'league_key': 683, 'league_name': 'UEFA Conference League', 'country_key': 1, 'country_name': 'eurocups', 'league_logo': None, 'country_logo': None}
 4. {'league_key': 3, 'league_name': 'UEFA Champions League', 'country_key': 1, 'country_name': 'eurocups', 'league_logo': 'https://apiv2.allsportsapi.com/logo/logo_leagues/3_uefa_champions_league.png', 'country_logo': None}
 5. {'league_key': 633, 'league_name': 'UEFA Nations League', 'country_key': 1, 'country_name': 'eurocups', 'league_logo': None, 'count

In [3]:
# What it does: Lists seasons for a league; TSDB primary (AllSports fallback returns Leagues raw).
# Edit leagueName or pass leagueId directly.

resp = call_router("seasons.list", {
    "leagueName": "English League Championship"
    # "leagueName": "Italian Serie A",
    # "leagueId": "4328"  # TSDB EPL id (example)
})
seasons = resp["data"].get("seasons") or resp["data"].get("result") or []
print("Seasons:", len(seasons))
show_first(seasons, n=10)

OK: True | Intent: seasons.list
Provider(s): {'primary': 'tsdb', 'fallback': None}
Seasons: 123
 1. {'strSeason': '1892-1893'}
 2. {'strSeason': '1893-1894'}
 3. {'strSeason': '1894-1895'}
 4. {'strSeason': '1895-1896'}
 5. {'strSeason': '1896-1897'}
 6. {'strSeason': '1897-1898'}
 7. {'strSeason': '1898-1899'}
 8. {'strSeason': '1899-1900'}
 9. {'strSeason': '1900-1901'}
10. {'strSeason': '1901-1902'}


In [4]:
# What it does: League detail; TSDB primary.
resp = call_router("league.get", {
    "leagueName": "Spanish La Liga",
    # "leagueId": "4335",  # TSDB La Liga (example)
})
print("Keys in data:", list(resp["data"].keys()))
show_first(resp["data"].get("league") or resp["data"])

OK: True | Intent: league.get
Provider(s): {'primary': 'tsdb', 'fallback': None}
Keys in data: ['league']
{'idLeague': '4335', 'idSoccerXML': None, 'idAPIfootball': '7351', 'strSport': 'Soccer', 'strLeague': 'Spanish La Liga', 'strLeagueAlternate': 'LaLiga Santander, La Liga', 'intDivision': '0', 'idCup': '0', 'strCurrentSeason': '2025-2026', 'intFormedYear': '1929', 'dateFirstEvent': '2003-01-05', 'strGender': 'Male'}


In [5]:
# What it does: Table/standings for a season; TSDB primary (AllSports Standings fallback).
# Change season to a valid one for that league.
resp = call_router("league.table", {
    "leagueName": "English Premier League",
    "season": "2014-2015",
    # AllSports fallback: "leagueId": "207", "season": "2020/2021"  # Serie A
})
table = resp["data"].get("table") or (resp["data"].get("result") or {}).get("total") or []
print("Rows:", len(table))
show_first(table, n=8)

OK: True | Intent: league.table
Provider(s): {'primary': 'tsdb', 'fallback': None}
Rows: 20
 1. {'idStanding': '244577', 'intRank': '1', 'idTeam': '133610', 'strTeam': 'Chelsea', 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/yvwvtu1448813215.png/tiny', 'idLeague': '4328', 'strLeague': 'English Premier League', 'strSeason': '2014-2015'}
 2. {'idStanding': '244578', 'intRank': '2', 'idTeam': '133613', 'strTeam': 'Manchester City', 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/vwpvry1467462651.png/tiny', 'idLeague': '4328', 'strLeague': 'English Premier League', 'strSeason': '2014-2015'}
 3. {'idStanding': '244579', 'intRank': '3', 'idTeam': '133604', 'strTeam': 'Arsenal', 'strBadge': 'https://r2.thesportsdb.com/images/media/team/badge/uyhbfe1612467038.png/tiny', 'idLeague': '4328', 'strLeague': 'English Premier League', 'strSeason': '2014-2015'}
 4. {'idStanding': '244580', 'intRank': '4', 'idTeam': '133612', 'strTeam': 'Manchester United', 'strBadge':

In [6]:
# What it does: Teams by league or by teamName; TSDB primary (AS fallback).
resp = call_router("teams.list", {
    "leagueName": "English Premier League",
    # or: "teamName": "Arsenal"
})
teams = resp["data"].get("teams") or resp["data"].get("result") or []
print("Teams:", len(teams))
show_first(teams, n=8)

OK: True | Intent: teams.list
Provider(s): {'primary': 'tsdb', 'fallback': None}
Teams: 20
 1. {'idTeam': '133604', 'idESPN': '359', 'idAPIfootball': '42', 'intLoved': '8', 'strTeam': 'Arsenal', 'strTeamAlternate': 'Arsenal Football Club, AFC, Arsenal FC', 'strTeamShort': 'ARS', 'intFormedYear': '1892'}
 2. {'idTeam': '133601', 'idESPN': '4328', 'idAPIfootball': '66', 'intLoved': None, 'strTeam': 'Aston Villa', 'strTeamAlternate': 'Aston Villa FC', 'strTeamShort': 'AVL', 'intFormedYear': '1874'}
 3. {'idTeam': '134301', 'idESPN': '349', 'idAPIfootball': '35', 'intLoved': '2', 'strTeam': 'Bournemouth', 'strTeamAlternate': 'AFC Bournemouth, Athletic Football Club Bournemouth', 'strTeamShort': 'BOU', 'intFormedYear': '1890'}
 4. {'idTeam': '134355', 'idESPN': '337', 'idAPIfootball': '55', 'intLoved': '1', 'strTeam': 'Brentford', 'strTeamAlternate': 'Brentford Football Club, Brentford FC, BFC', 'strTeamShort': 'BRE', 'intFormedYear': '1889'}
 5. {'idTeam': '133619', 'idESPN': '331', 'idAPI

In [7]:
# What it does: Team detail; TSDB primary; AS fallback returns Teams result.
resp = call_router("team.get", {
    # "teamName": "Juventus",
    # "teamId": "133676"  # TSDB Juventus (example)
    # AllSports: "teamId": "96"
})
show_first(resp["data"].get("team") or resp["data"])

OK: True | Intent: team.get
Provider(s): {'primary': 'tsdb', 'fallback': 'allsports'}
{'error': '1', 'result': [{'param': 'teamId', 'msg': 'Required parameter missing', 'cod': 201}]}


In [8]:
# What it does: Team equipment/kit images; TSDB-only feature.
resp = call_router("team.equipment", {
    "teamName": "Juventus",
    # "teamId": "133604"   # TSDB Arsenal (example)
})
equip = resp["data"].get("equipment") or []
print("Equipment rows:", len(equip))
show_first(equip, n=6)

OK: True | Intent: team.equipment
Provider(s): {'primary': 'tsdb', 'fallback': None}
Equipment rows: 44
 1. {'idEquipment': '795', 'idTeam': '133676', 'date': '2019-10-09 14:41:44', 'strSeason': '2019-2020', 'strEquipment': 'https://r2.thesportsdb.com/images/media/team/equipment/2019-133676-Jersey.png', 'strType': '1st', 'strUsername': 'smudgie'}
 2. {'idEquipment': '1152', 'idTeam': '133676', 'date': '2020-04-15 22:10:38', 'strSeason': '2019-2020', 'strEquipment': 'https://r2.thesportsdb.com/images/media/team/equipment/1yctl81586985025.png', 'strType': '2nd', 'strUsername': 'smudgie'}
 3. {'idEquipment': '1153', 'idTeam': '133676', 'date': '2020-04-15 22:10:55', 'strSeason': '2019-2020', 'strEquipment': 'https://r2.thesportsdb.com/images/media/team/equipment/newfxj1586985041.png', 'strType': '3rd', 'strUsername': 'smudgie'}
 4. {'idEquipment': '1687', 'idTeam': '133676', 'date': '2020-09-08 14:30:37', 'strSeason': '2020', 'strEquipment': 'https://r2.thesportsdb.com/images/media/team/e

In [9]:
# What it does: Players; AllSports primary (supports playerName/teamId/leagueId), TSDB fallback if empty.
# Change teamName/teamId as needed. Example AS teamId: Juventus=96
resp = call_router("players.list", {
    # By team:
    # "teamName": "Juventus",
    # "teamId": "96",             # AllSports Juventus
    # Or by player:
    # "playerName": "Lionel Messi",
})
players = resp["data"].get("result") or resp["data"].get("players") or []
print("Players:", len(players))
show_first(players, n=8)

OK: True | Intent: players.list
Provider(s): {'primary': 'allsports', 'fallback': None}
Players: 1
 1. {'param': 'playerId or playerName', 'msg': 'Required parameter missing', 'cod': 201}


In [10]:
# What it does: Player detail; AllSports primary, TSDB fallback.
resp = call_router("player.get", {
    "playerName": "C. Pinsoglio",
    # AllSports: "playerId": 103051168,
})
out = resp["data"].get("result") or resp["data"].get("players") or resp["data"]
if isinstance(out, list) and out:
    show_first(out, n=3)
else:
    show_first(out)

OK: True | Intent: player.get
Provider(s): {'primary': 'allsports', 'fallback': None}
 1. {'player_key': 1136721488, 'player_name': 'C. Pinsoglio', 'player_number': '23', 'player_country': 'Italy', 'player_type': 'Goalkeepers', 'player_age': '35', 'player_match_played': '', 'player_goals': ''}


In [11]:
# What it does: TSDB player extras (if available on free tier).
resp = call_router("player.honours", {"playerName": "Lionel Messi"})
print("Honours keys:", list(resp["data"].keys()))
show_first(resp["data"].get("honours") or [])

OK: True | Intent: player.honours
Provider(s): {'primary': 'tsdb', 'fallback': None}
Honours keys: ['honours', 'count']
 1. {'id': '611', 'idPlayer': '34146370', 'idTeam': '133739', 'idLeague': '4335', 'idHonour': '566', 'strSport': 'Soccer', 'strPlayer': 'Lionel Messi', 'strTeam': 'Barcelona'}
 2. {'id': '612', 'idPlayer': '34146370', 'idTeam': '133739', 'idLeague': '4335', 'idHonour': '566', 'strSport': 'Soccer', 'strPlayer': 'Lionel Messi', 'strTeam': 'Barcelona'}
 3. {'id': '613', 'idPlayer': '34146370', 'idTeam': '133739', 'idLeague': '4335', 'idHonour': '566', 'strSport': 'Soccer', 'strPlayer': 'Lionel Messi', 'strTeam': 'Barcelona'}


In [12]:
# What it does: Match list by day; TSDB primary, AS fallback if empty.
resp = call_router("events.list", {
    "date": "2014-12-29",
    # try more: "2021-05-18", "2023-12-16"
})
events = resp["data"].get("events") or resp["data"].get("result") or resp["data"].get("results") or []
print("Events:", len(events))
show_first(events, n=8)

OK: True | Intent: events.list
Provider(s): {'primary': 'tsdb', 'fallback': None}
Events: 1
 1. {'idEvent': '441043', 'idAPIfootball': None, 'strEvent': 'Melbourne Victory vs Western Sydney Wanderers FC', 'strEventAlternate': 'Western Sydney Wanderers FC @ Melbourne Victory', 'strFilename': 'Australian A-League 2014-10-10 Melbourne Victory vs Western Sydney Wanderers FC', 'strSport': 'Soccer', 'idLeague': '4356', 'strLeague': 'Australian A-League'}


In [13]:
# What it does: Matches for a league season; TSDB primary; AS fallback via Fixtures window if needed.
resp = call_router("events.list", {
    "leagueName": "Belgian Pro League",
    "season": "2014-2015",
})
events = resp["data"].get("events") or resp["data"].get("result") or []
print("Events:", len(events))
show_first(events, n=5)

OK: True | Intent: events.list
Provider(s): {'primary': 'tsdb', 'fallback': None}
Events: 100
 1. {'idEvent': '444581', 'idAPIfootball': None, 'strEvent': 'Standard vs Charleroi', 'strEventAlternate': 'Charleroi @ Standard', 'strFilename': 'Belgian Jupiler League 2014-07-25 Standard vs Charleroi', 'strSport': 'Soccer', 'idLeague': '4338', 'strLeague': 'Belgian Jupiler League'}
 2. {'idEvent': '444582', 'idAPIfootball': None, 'strEvent': 'Waasland-Beveren vs Club Brugge', 'strEventAlternate': 'Club Brugge @ Waasland-Beveren', 'strFilename': 'Belgian Jupiler League 2014-07-26 Waasland-Beveren vs Club Brugge', 'strSport': 'Soccer', 'idLeague': '4338', 'strLeague': 'Belgian Jupiler League'}
 3. {'idEvent': '444583', 'idAPIfootball': None, 'strEvent': 'Westerlo vs Lokeren', 'strEventAlternate': 'Lokeren @ Westerlo', 'strFilename': 'Belgian Jupiler League 2014-07-26 Westerlo vs Lokeren', 'strSport': 'Soccer', 'idLeague': '4338', 'strLeague': 'Belgian Jupiler League'}
 4. {'idEvent': '444584'

In [14]:
# What it does: Matches for a team; TSDB primary (next/last); AS fallback via Fixtures teamId window.
resp = call_router("events.list", {
    "teamName": "Arsenal",
    # "kind": "next",  # TSDB: next matches for team
    # Default is "last" on TSDB team path
})
events = resp["data"].get("results") or resp["data"].get("events") or resp["data"].get("result") or []
print("Team events:", len(events))
show_first(events, n=6)

OK: True | Intent: events.list
Provider(s): {'primary': 'tsdb', 'fallback': None}
Team events: 5
 1. {'idEvent': '2267083', 'idAPIfootball': '1378979', 'strEvent': 'Arsenal vs Leeds United', 'strEventAlternate': 'Leeds United @ Arsenal', 'strFilename': 'English Premier League 2025-08-23 Arsenal vs Leeds United', 'strSport': 'Soccer', 'idLeague': '4328', 'strLeague': 'English Premier League'}
 2. {'idEvent': '2290379', 'idAPIfootball': '1430657', 'strEvent': 'Arsenal vs Athletic Bilbao', 'strEventAlternate': 'Athletic Bilbao @ Arsenal', 'strFilename': 'Emirates Cup 2025-08-09 Arsenal vs Athletic Bilbao', 'strSport': 'Soccer', 'idLeague': '4569', 'strLeague': 'Club Friendlies'}
 3. {'idEvent': '2290369', 'idAPIfootball': '1402825', 'strEvent': 'Arsenal vs Villarreal', 'strEventAlternate': 'Villarreal @ Arsenal', 'strFilename': 'Club Friendlies 2025-08-06 Arsenal vs Villarreal', 'strSport': 'Soccer', 'idLeague': '4569', 'strLeague': 'Club Friendlies'}
 4. {'idEvent': '2250034', 'idAPIfoot

In [15]:
# What it does: TSDB **name-first** event detail; keeps candidates, chooses 1 if unique or by matching id; expands.
resp = call_router("event.get", {
    "eventName": "Standard vs Charleroi",
    # "eventId": "444581",             # optional filter on candidates
    "expand": ["timeline", "stats", "lineup"]
})
data = resp["data"]
print("candidates:", len(data.get("candidates") or []))
print("picked id:", (data.get("event") or {}).get("idEvent"))
print("expansion keys:", [k for k in ("timeline","stats","lineup") if k in data])

OK: True | Intent: event.get
Provider(s): {'primary': 'tsdb', 'fallback': None}
candidates: 14
picked id: None
expansion keys: []


In [16]:
# What it does: Live matches; AllSports primary; TSDB fallback is limited.
resp = call_router("events.live", {
    # Optional: "leagueName": "Italian Serie A", "countryName": "Italy",
    # Optional: "withPlayerStats": 1
})
live = resp["data"].get("result") or resp["data"].get("events") or []
print("Live count:", len(live))
show_first(live, n=5)

OK: True | Intent: events.live
Provider(s): {'primary': 'allsports', 'fallback': 'tsdb'}
Live count: 0


In [17]:
# What it does: Highlights; TSDB primary, AllSports fallback
resp = call_router("video.highlights", {
    # If you have an eventId, include it:
    # "eventId": "528603",
    # Otherwise TSDB will try to use the chosen event from name-based get flows.
    # "eventName": "Liverpool vs Swansea"
})
print("Keys:", list(resp["data"].keys()))
show_first(resp["data"].get("result") or resp["data"].get("highlights") or resp["data"])

OK: True | Intent: video.highlights
Provider(s): {'primary': 'allsports', 'fallback': 'tsdb'}
Keys: ['success']
{'success': 1}


In [18]:
# What it does: Odds (AllSports only). Use either a date window or a matchId.
resp = call_router("odds.list", {
    # By match:
    # "matchId": "86392",
    # OR date window:
    # "from": "2021-05-18", "to": "2021-05-18"
})
print("Keys:", list(resp["data"].keys()))
show_first(resp["data"].get("result") or resp["data"])

OK: True | Intent: odds.list
Provider(s): {'primary': 'allsports', 'fallback': None}
Keys: ['error', 'result']
 1. {'param': 'from', 'msg': 'Required parameter missing', 'cod': 201}
 2. {'param': 'to', 'msg': 'Required parameter missing', 'cod': 201}


In [1]:
# --- Robust project locator for "sports-ai" ---
import sys, os
from pathlib import Path

# Start from the notebook location and walk up to find /sports-ai/backend/app
here = Path.cwd()
sports_ai_root = None

for p in [here, *here.parents]:
    # Case A: repo layout like .../Sports Analysis/sports-ai/backend/app
    if (p / "sports-ai" / "backend" / "app").exists():
        sports_ai_root = p / "sports-ai"
        break
    # Case B: notebook already inside sports-ai/
    if (p / "backend" / "app").exists() and (p.name == "sports-ai"):
        sports_ai_root = p
        break

if not sports_ai_root:
    raise RuntimeError(
        "Could not find 'sports-ai/backend/app'. "
        "Run the notebook from your repo or update the search logic."
    )

# Put sports-ai/ and sports-ai/backend/ on sys.path
sys.path.insert(0, str(sports_ai_root))
sys.path.insert(0, str(sports_ai_root / "backend"))

print("Using sports_ai_root:", sports_ai_root)
print("Top of sys.path:", sys.path[:3])

Using sports_ai_root: /Users/praise/Documents/GitHub/Sports Analysis/sports-ai
Top of sys.path: ['/Users/praise/Documents/GitHub/Sports Analysis/sports-ai/backend', '/Users/praise/Documents/GitHub/Sports Analysis/sports-ai', '/opt/anaconda3/lib/python312.zip']


In [10]:
# --- Locate project & import RouterCollector (AllSports is the default provider for these intents) ---
import sys, os
from pathlib import Path

here = Path.cwd()
sports_ai_root = None
for p in [here, *here.parents]:
    if (p / "sports-ai" / "backend" / "app").exists():
        sports_ai_root = p / "sports-ai"
        break
    if (p / "backend" / "app").exists() and p.name == "sports-ai":
        sports_ai_root = p
        break

if not sports_ai_root:
    raise RuntimeError("Could not find 'sports-ai/backend/app' from this notebook location.")

sys.path.insert(0, str(sports_ai_root))
sys.path.insert(0, str(sports_ai_root / "backend"))

from backend.app.routers.router_collector import RouterCollector
router = RouterCollector()

# Small helper to normalize provider payload lists
def _pluck_result(d):
    if not isinstance(d, dict): return d
    for k in ("result","results","events","fixtures","list"):
        v = d.get(k)
        if isinstance(v, list): return v
    return d

In [11]:
import pandas as pd

resp = router.handle({"intent": "leagues.list", "args": {}})
leagues = _pluck_result(resp.get("data") or {}) or []
df_leagues = pd.DataFrame(leagues)

# Show the most useful ID columns up front
wanted = [c for c in ["league_key","league_name","country_key","country_name","league_season"] if c in df_leagues.columns]
df_leagues[wanted].head(30)

,league_key,league_name,country_key,country_name
0,4,UEFA Europa League,1,eurocups
1,1,UEFA European Championship,1,eurocups
2,683,UEFA Conference League,1,eurocups
3,3,UEFA Champions League,1,eurocups
4,633,UEFA Nations League,1,eurocups
5,28,World Cup,8,Worldcup
6,152,Premier League,44,England
7,302,La Liga,6,Spain
8,207,Serie A,5,Italy
9,175,Bundesliga,4,Germany


In [12]:
from datetime import datetime, timedelta
import pandas as pd

LEAGUE_ID = "152"  # <-- set the league ID you want (string or int is fine)

today = datetime.utcnow().date()
args = {
    "leagueId": str(LEAGUE_ID),
    "from": (today - timedelta(days=7)).isoformat(),
    "to": today.isoformat(),
    # Optional: uncomment the next line ONLY if your router supports provider pinning
    # "provider": "allsports"
}
resp = router.handle({"intent": "events.list", "args": args})
matches = _pluck_result(resp.get("data") or {}) or []

def pick(k, d, alt=()):
    for kk in (k, *alt):
        if kk in d: return d[kk]
    return None

rows = [{
    "date":     pick("event_date", m, ("dateEvent",)),
    "time":     pick("event_time", m, ("strTime",)),
    "home":     pick("event_home_team", m, ("strHomeTeam","homeTeam")),
    "away":     pick("event_away_team", m, ("strAwayTeam","awayTeam")),
    "status":   pick("event_status", m, ("strStatus","status")),
    "HS":       pick("home_score", m, ("intHomeScore","home_score_ft")),
    "AS":       pick("away_score", m, ("intAwayScore","away_score_ft")),
    "event_id": pick("event_key", m, ("idEvent","id")),
} for m in matches]

df_matches = pd.DataFrame(rows)
if {"date","time"} <= set(df_matches.columns):
    df_matches = df_matches.sort_values(["date","time"], ascending=[False, False])

print(f"League {LEAGUE_ID}: {len(df_matches)} matches")
df_matches.head(30)

/var/folders/78/1x37jfss4rx38yjhktq5fgx00000gn/T/ipykernel_34749/3033725522.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = datetime.utcnow().date()


League 152: 10 matches


,date,time,home,away,status,HS,AS,event_id
0,2025-08-31,20:00,Aston Villa,Crystal Palace,Finished,None,None,1626154
3,2025-08-31,17:30,Liverpool,Arsenal,Finished,None,None,1626163
1,2025-08-31,15:00,Nottingham,West Ham,Finished,None,None,1626161
2,2025-08-31,15:00,Brighton,Manchester City,Finished,None,None,1626162
9,2025-08-30,18:30,Leeds,Newcastle,Finished,None,None,1626160
4,2025-08-30,16:00,Sunderland,Brentford,Finished,None,None,1626155
6,2025-08-30,16:00,Manchester Utd,Burnley,Finished,None,None,1626157
7,2025-08-30,16:00,Tottenham,Bournemouth,Finished,None,None,1626158
8,2025-08-30,16:00,Wolves,Everton,Finished,None,None,1626159
5,2025-08-30,13:30,Chelsea,Fulham,Finished,None,None,1626156


In [5]:
import requests, json

API_BASE = "http://localhost:8000"  # 🔁 change to your backend URL if deployed

def call_router(intent, args=None):
    """POST helper to hit your unified agent router."""
    r = requests.post(f"{API_BASE}/collect",
                      json={"intent": intent, "args": args or {}},
                      timeout=30)
    print("HTTP:", r.status_code)
    resp = r.json()
    if not resp.get("ok"):
        raise RuntimeError(json.dumps(resp, indent=2))
    return resp

def show_preview(items, max_items=10):
    """Pretty-print up to N items."""
    print(f"Items: {len(items)} (showing {max_items})\n")
    for i, x in enumerate(items[:max_items], 1):
        print(f"{i:02}.", x.get("league_name") or x.get("strLeague"),
              "-", x.get("country_name") or x.get("strCountry"))

In [6]:
# Request all leagues where country_name == "England"
resp = call_router("leagues.list", {})
data = resp["data"]

# AllSports shape usually has 'result' list
leagues = data.get("result") or data.get("leagues") or []

# Filter client-side, since AllSports API does not support country filter directly
eng_leagues = [
    lg for lg in leagues
    if (lg.get("country_name") or lg.get("strCountry") or "").lower() == "england"
]

show_preview(eng_leagues)

HTTP: 200
Items: 26 (showing 10)

01. Premier League - England
02. EFL Trophy - England
03. League Two - England
04. FA Cup - England
05. League Cup - England
06. Championship - England
07. League One - England
08. Community Shield - England
09. Non League Div One - England
10. FA Trophy - England


In [1]:
# Setup: HTTP caller + helpers
import requests, json, pandas as pd

AGENT_URL = "http://127.0.0.1:8000/collect"  # change if your server runs elsewhere

def call_router(intent: str, args: dict | None = None):
    r = requests.post(AGENT_URL, json={"intent": intent, "args": args or {}}, timeout=60)
    r.raise_for_status()
    resp = r.json()
    print("OK:", resp.get("ok"), "| Intent:", resp.get("intent"))
    return resp

def pluck_list(d: dict):
    if not isinstance(d, dict):
        return []
    for k in ("leagues", "result", "results", "list"):
        v = d.get(k)
        if isinstance(v, list):
            return v
    return []

# Fetch and display league list
resp = call_router("leagues.list", {})
data = resp.get("data") or {}
leagues = pluck_list(data)

print("Total leagues:", len(leagues))
df = pd.DataFrame(leagues)
cols = [c for c in ["league_key","league_name","country_key","country_name","league_season","league_logo"] if c in df.columns]
display(df[cols].head(50))

# Optional: show a few raw JSON rows
print(json.dumps(leagues[:5], indent=2) if leagues else "[]")

OK: True | Intent: leagues.list
Total leagues: 976


,league_key,league_name,country_key,country_name,league_logo
0,4,UEFA Europa League,1,eurocups,https://apiv2.allsportsapi.com/logo/logo_leagues/
1,1,UEFA European Championship,1,eurocups,None
2,683,UEFA Conference League,1,eurocups,None
3,3,UEFA Champions League,1,eurocups,https://apiv2.allsportsapi.com/logo/logo_leagu...
4,633,UEFA Nations League,1,eurocups,None
5,28,World Cup,8,Worldcup,https://apiv2.allsportsapi.com/logo/logo_leagu...
6,152,Premier League,44,England,https://apiv2.allsportsapi.com/logo/logo_leagu...
7,302,La Liga,6,Spain,https://apiv2.allsportsapi.com/logo/logo_leagu...
8,207,Serie A,5,Italy,https://apiv2.allsportsapi.com/logo/logo_leagu...
9,175,Bundesliga,4,Germany,https://apiv2.allsportsapi.com/logo/logo_leagu...


[
  {
    "league_key": 4,
    "league_name": "UEFA Europa League",
    "country_key": 1,
    "country_name": "eurocups",
    "league_logo": "https://apiv2.allsportsapi.com/logo/logo_leagues/",
    "country_logo": null
  },
  {
    "league_key": 1,
    "league_name": "UEFA European Championship",
    "country_key": 1,
    "country_name": "eurocups",
    "league_logo": null,
    "country_logo": null
  },
  {
    "league_key": 683,
    "league_name": "UEFA Conference League",
    "country_key": 1,
    "country_name": "eurocups",
    "league_logo": null,
    "country_logo": null
  },
  {
    "league_key": 3,
    "league_name": "UEFA Champions League",
    "country_key": 1,
    "country_name": "eurocups",
    "league_logo": "https://apiv2.allsportsapi.com/logo/logo_leagues/3_uefa_champions_league.png",
    "country_logo": null
  },
  {
    "league_key": 633,
    "league_name": "UEFA Nations League",
    "country_key": 1,
    "country_name": "eurocups",
    "league_logo": null,
    "country

In [2]:
# ✅ Export only league_name and country_name to CSV
export_cols = ["league_name", "country_name"]
csv_df = df[export_cols].drop_duplicates()

# Save to CSV
csv_filename = "leagues_table.csv"
csv_df.to_csv(csv_filename, index=False, encoding="utf-8")

print(f"✅ CSV file saved as {csv_filename} with {len(csv_df)} rows")

✅ CSV file saved as leagues_table.csv with 945 rows


In [8]:
# Simple filter for top-5 domestic leagues by country + name patterns
import re, json
def norm(s): return re.sub(r"[^a-z0-9]+", "", str(s).lower())

def is_top5(row):
    c = norm(row.get("country_name", ""))
    n = norm(row.get("league_name", ""))
    if c in ("england","english"):
        return "premierleague" in n or "englishpremierleague" in n
    if c == "spain":
        return ("laliga" in n) or ("primeradivision" in n) or ("laligasantander" in n)
    if c == "italy":
        return "seriea" in n
    if c == "germany":
        return "bundesliga" in n
    if c == "france":
        return "ligue1" in n
    return False

df_top = df[df.apply(is_top5, axis=1)].copy()
cols_top = [c for c in ["league_key","league_name","country_name","league_season","league_logo"] if c in df_top.columns]
print("Top European leagues found:", len(df_top))
display(df_top[cols_top].sort_values(["country_name","league_name"]))

# Also show the exact JSON rows (truncated)
top_json = df_top.to_dict(orient="records")
print(json.dumps(top_json[:10], indent=2))

Top European leagues found: 14


,league_key,league_name,country_name,league_logo
6,152,Premier League,England,https://apiv2.allsportsapi.com/logo/logo_leagu...
594,593,Premier League 2,England,https://apiv2.allsportsapi.com/logo/logo_leagu...
710,716,Premier League Cup,England,None
801,10224,Premier League Summer Series,England,None
622,622,U18 Premier League,England,https://apiv2.allsportsapi.com/logo/logo_leagu...
10,168,Ligue 1,France,https://apiv2.allsportsapi.com/logo/logo_leagu...
41,171,2. Bundesliga,Germany,https://apiv2.allsportsapi.com/logo/logo_leagu...
786,10209,2. Bundesliga Women,Germany,None
9,175,Bundesliga,Germany,https://apiv2.allsportsapi.com/logo/logo_leagu...
116,7,Frauen Bundesliga,Germany,https://apiv2.allsportsapi.com/logo/logo_leagu...


[
  {
    "league_key": 152,
    "league_name": "Premier League",
    "country_key": 44,
    "country_name": "England",
    "league_logo": "https://apiv2.allsportsapi.com/logo/logo_leagues/152_premier-league.png",
    "country_logo": "https://apiv2.allsportsapi.com/logo/logo_country/44_england.png"
  },
  {
    "league_key": 302,
    "league_name": "La Liga",
    "country_key": 6,
    "country_name": "Spain",
    "league_logo": "https://apiv2.allsportsapi.com/logo/logo_leagues/302_la-liga.png",
    "country_logo": "https://apiv2.allsportsapi.com/logo/logo_country/6_spain.png"
  },
  {
    "league_key": 207,
    "league_name": "Serie A",
    "country_key": 5,
    "country_name": "Italy",
    "league_logo": "https://apiv2.allsportsapi.com/logo/logo_leagues/207_serie-a.png",
    "country_logo": "https://apiv2.allsportsapi.com/logo/logo_country/5_italy.png"
  },
  {
    "league_key": 175,
    "league_name": "Bundesliga",
    "country_key": 4,
    "country_name": "Germany",
    "league_logo

In [9]:
# Filter leagues by England (works across provider shapes)
resp = call_router("leagues.list", {})  # Tip: try {"country": "England"} to let TSDB filter server-side
data = resp.get("data") or {}
leagues = data.get("leagues") or data.get("result") or data.get("results") or []

def get_country(row):
    return (row.get("country_name") or row.get("strCountry") or row.get("country") or "").strip()

eng = [row for row in leagues if get_country(row).lower() in ("england", "english")]

print("England leagues:", len(eng))

import pandas as pd, json
df_eng = pd.DataFrame(eng)
cols = [c for c in ["league_key","league_name","country_name","league_season","league_logo"] if c in df_eng.columns]
display(df_eng[cols].sort_values(["league_name"]).head(50))

# Optional: show raw JSON rows
print(json.dumps(eng[:10], indent=2))

OK: True | Intent: leagues.list
England leagues: 26


,league_key,league_name,country_name,league_logo
5,153,Championship,England,https://apiv2.allsportsapi.com/logo/logo_leagu...
7,377,Community Shield,England,https://apiv2.allsportsapi.com/logo/logo_leagu...
1,148,EFL Trophy,England,https://apiv2.allsportsapi.com/logo/logo_leagu...
3,146,FA Cup,England,https://apiv2.allsportsapi.com/logo/logo_leagu...
9,144,FA Trophy,England,https://apiv2.allsportsapi.com/logo/logo_leagu...
20,655,FA Women's League Cup,England,None
24,10794,FA Youth Cup,England,None
4,147,League Cup,England,https://apiv2.allsportsapi.com/logo/logo_leagu...
6,154,League One,England,https://apiv2.allsportsapi.com/logo/logo_leagu...
2,145,League Two,England,https://apiv2.allsportsapi.com/logo/logo_leagu...


[
  {
    "league_key": 152,
    "league_name": "Premier League",
    "country_key": 44,
    "country_name": "England",
    "league_logo": "https://apiv2.allsportsapi.com/logo/logo_leagues/152_premier-league.png",
    "country_logo": "https://apiv2.allsportsapi.com/logo/logo_country/44_england.png"
  },
  {
    "league_key": 148,
    "league_name": "EFL Trophy",
    "country_key": 44,
    "country_name": "England",
    "league_logo": "https://apiv2.allsportsapi.com/logo/logo_leagues/148_efl-trophy.png",
    "country_logo": "https://apiv2.allsportsapi.com/logo/logo_country/44_england.png"
  },
  {
    "league_key": 145,
    "league_name": "League Two",
    "country_key": 44,
    "country_name": "England",
    "league_logo": "https://apiv2.allsportsapi.com/logo/logo_leagues/145_league-two.png",
    "country_logo": "https://apiv2.allsportsapi.com/logo/logo_country/44_england.png"
  },
  {
    "league_key": 146,
    "league_name": "FA Cup",
    "country_key": 44,
    "country_name": "Engla

In [10]:
# Show all leagues whose name contains "Premier League"
resp = call_router("leagues.list", {})
data = resp.get("data") or {}
leagues = data.get("leagues") or data.get("result") or data.get("results") or []

def get_name(row):
    return (row.get("league_name") or row.get("strLeague") or "").strip()

premier = [row for row in leagues if "premier league" in get_name(row).lower()]

print("Premier League-like rows:", len(premier))

# Pretty print just names (and country)
for r in premier:
    print("-", get_name(r), "|", (r.get("country_name") or r.get("strCountry") or "").strip())

# Optional: DataFrame view
import pandas as pd, json
df_pl = pd.DataFrame(premier)
cols = [c for c in ["league_key","league_name","country_name","league_season","league_logo","country_key"] if c in df_pl.columns]
if not df_pl.empty and cols:
    display(df_pl[cols].sort_values(["country_name","league_name"]))

# Optional: raw JSON preview (first 10)
print(json.dumps(premier[:10], indent=2))

OK: True | Intent: leagues.list
Premier League-like rows: 54
- Premier League | England
- Premier League | Russia
- Premier League | Ukraine
- Premier League | Kazakhstan
- Premier League | Belarus
- Premier League | Egypt
- Premier League | Armenia
- Premier League | Bahrain
- Premier League | Bangladesh
- Premier League | Bermuda
- Taiwan Football Premier League | Chinese Taipei
- Premier League | Ghana
- Premier League | Hong Kong
- Premier League | Jamaica
- FKF Premier League | Kenya
- Premier League | Kuwait
- Premier League | Lebanon
- Premier League | Libya
- Premier League | Malaysia
- Premier League | Malta
- Premier League | Singapore
- Sudani Premier League | Sudan
- Premier League | Syria
- TT Premier League | Trinidad and Tobago
- Premier League | Wales
- Northern Territory Premier League | Australia
- Premier League | Barbados
- Premier League | Mongolia
- Premier League | Belize
- Premier League | Botswana
- Premier League | Cambodia
- Premier League Asia Trophy | intl


,league_key,league_name,country_name,league_logo,country_key
6,45,Premier League,Armenia,https://apiv2.allsportsapi.com/logo/logo_leagu...,15
25,373,Northern Territory Premier League,Australia,https://apiv2.allsportsapi.com/logo/logo_leagu...,17
45,678,Queensland Premier League,Australia,None,17
7,59,Premier League,Bahrain,https://apiv2.allsportsapi.com/logo/logo_leagu...,20
8,60,Premier League,Bangladesh,https://apiv2.allsportsapi.com/logo/logo_leagu...,21
26,403,Premier League,Barbados,None,125
53,10610,Premier League,Barbados,None,125
4,61,Premier League,Belarus,https://apiv2.allsportsapi.com/logo/logo_leagu...,22
28,488,Premier League,Belize,None,134
9,68,Premier League,Bermuda,https://apiv2.allsportsapi.com/logo/logo_leagu...,24


[
  {
    "league_key": 152,
    "league_name": "Premier League",
    "country_key": 44,
    "country_name": "England",
    "league_logo": "https://apiv2.allsportsapi.com/logo/logo_leagues/152_premier-league.png",
    "country_logo": "https://apiv2.allsportsapi.com/logo/logo_country/44_england.png"
  },
  {
    "league_key": 344,
    "league_name": "Premier League",
    "country_key": 95,
    "country_name": "Russia",
    "league_logo": "https://apiv2.allsportsapi.com/logo/logo_leagues/344_premier-league.png",
    "country_logo": "https://apiv2.allsportsapi.com/logo/logo_country/95_russia.png"
  },
  {
    "league_key": 325,
    "league_name": "Premier League",
    "country_key": 112,
    "country_name": "Ukraine",
    "league_logo": "https://apiv2.allsportsapi.com/logo/logo_leagues/325_premier-league.png",
    "country_logo": "https://apiv2.allsportsapi.com/logo/logo_country/112_ukraine.png"
  },
  {
    "league_key": 214,
    "league_name": "Premier League",
    "country_key": 66,
  